# Sydney Green Space Detection
## Training Random Forest with WorldCover 2021 as Ground Truth

**Key Features:**
- Uses **WorldCover 2021** as ground truth for training
- Green classes: Tree cover (10), Shrubland (20), Grassland (30), Mangroves (95)
- Multi-temporal Sentinel-2 data (April, August, November)
- 21 bands: 4 spectral bands × 3 months + 3 vegetation indices × 3 months

## 1. Import Libraries

In [ ]:
import json
import os
import glob
import numpy as np
import rasterio
from pathlib import Path
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## 2. Configuration - Sydney Paths

In [ ]:
# Base paths
BASE_PATH = "/Users/timgotschim/Documents/LLM/infrared.city"
CITY = "Sydney"

# Input paths
aoi_file = os.path.join(BASE_PATH, "aois_json/Sydney.geojson")
output_folder = os.path.join(BASE_PATH, "sentinel_data/Sydney")
os.makedirs(output_folder, exist_ok=True)

# Sentinel-2 data folders
sentinel_folders = {
    "April": "/Users/timgotschim/Documents/LLM/AOI_10m/Sydney_APR_R10m",
    "August": "/Users/timgotschim/Documents/LLM/AOI_10m/Sydney_AUG_10m",
    "November": "/Users/timgotschim/Documents/LLM/AOI_10m/Sydney_NOV_10m"
}

# WorldCover path - CHANGE THIS TO YOUR WORLDCOVER FILE
worldcover_file = os.path.join(BASE_PATH, "worldcover/Sydney_WorldCover_2021.tif")

# Bands to process
band_substrings = ["B02", "B03", "B04", "B08"]

# Output files
stack_file = os.path.join(output_folder, "Sydney_MultiMonth_stack.tif")
worldcover_labels_file = os.path.join(output_folder, "Sydney_WorldCover_labels.tif")

# Create results folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_folder = os.path.join(output_folder, f"run_{timestamp}")
os.makedirs(run_folder, exist_ok=True)

print("✓ Configuration loaded")
print(f"  City: {CITY}")
print(f"  Output folder: {output_folder}")
print(f"  Results folder: {run_folder}")

## 3. Create Multi-Month Stack (21 Bands)
### Load and clip Sentinel-2 bands, calculate vegetation indices

In [ ]:
print("="*60)
print(f"PROCESSING {CITY} - Creating 21-band stack")
print("="*60)

# Load AOI
aoi = gpd.read_file(aoi_file)
if len(aoi) > 1:
    merged_geom = aoi.unary_union
    geometries = [merged_geom]
else:
    geometries = [aoi.geometry.iloc[0]]

# Ensure WGS84
if aoi.crs is None:
    aoi.set_crs("EPSG:4326", inplace=True)
if aoi.crs.to_epsg() != 4326:
    geometries = [g.to_crs("EPSG:4326") for g in geometries]

print(f"Loaded AOI for {CITY}")
print(f"AOI bounds: {aoi.total_bounds}")

all_band_arrays = []
all_band_names = []

# Process each month
for month, folder_path in sentinel_folders.items():
    print(f"\n=== Processing {month} ===")
    
    if not os.path.exists(folder_path):
        print(f"WARNING: Folder not found: {folder_path}")
        continue
    
    month_band_dict = {}
    
    # Load each band
    for substring in band_substrings:
        matched_files = glob.glob(os.path.join(folder_path, f"*{substring}*10m.jp2"))
        if not matched_files:
            print(f"WARNING: No file found for band '{substring}' in {folder_path}")
            continue
        
        band_path = matched_files[0]
        print(f"Loading: {os.path.basename(band_path)}")
        
        band = rxr.open_rasterio(band_path, masked=True).squeeze()
        band_clipped = band.rio.clip(geometries, crs="EPSG:4326")
        
        band_name = f"{substring}-{month}"
        all_band_arrays.append(band_clipped)
        all_band_names.append(band_name)
        month_band_dict[substring] = band_clipped
        
        print(f"  Clipped {band_name} -> shape: {band_clipped.shape}")
    
    # Calculate vegetation indices
    if len(month_band_dict) >= 3:
        # NDVI
        if "B08" in month_band_dict and "B04" in month_band_dict:
            nir = month_band_dict["B08"].astype(np.float32)
            red = month_band_dict["B04"].astype(np.float32)
            
            ndvi = (nir - red) / (nir + red)
            ndvi = xr.where(np.isfinite(ndvi), ndvi, np.nan)
            ndvi_name = f"NDVI-{month}"
            all_band_arrays.append(ndvi)
            all_band_names.append(ndvi_name)
            print(f"  Calculated {ndvi_name} -> range: [{float(ndvi.min()):.3f}, {float(ndvi.max()):.3f}]")
        
        # EVI
        if "B08" in month_band_dict and "B04" in month_band_dict and "B02" in month_band_dict:
            blue = month_band_dict["B02"].astype(np.float32)
            
            evi = 2.5 * (nir - red) / (nir + 6*red - 7.5*blue + 1)
            evi = xr.where(np.isfinite(evi), evi, np.nan)
            evi_name = f"EVI-{month}"
            all_band_arrays.append(evi)
            all_band_names.append(evi_name)
            print(f"  Calculated {evi_name} -> range: [{float(evi.min()):.3f}, {float(evi.max()):.3f}]")
        
        # SAVI
        if "B08" in month_band_dict and "B04" in month_band_dict:
            L = 0.5
            savi = ((nir - red) * (1 + L)) / (nir + red + L)
            savi = xr.where(np.isfinite(savi), savi, np.nan)
            savi_name = f"SAVI-{month}"
            all_band_arrays.append(savi)
            all_band_names.append(savi_name)
            print(f"  Calculated {savi_name} -> range: [{float(savi.min()):.3f}, {float(savi.max()):.3f}]")

# Stack all bands
print(f"\n=== Creating final stack ===")
stack = xr.concat(all_band_arrays, dim="band")
stack = stack.assign_coords(band=all_band_names)
stack = stack.astype(np.float32)

print(f"Stacked all bands -> shape: {stack.shape}")
print(f"Total bands: {len(all_band_names)}")
print(f"Band order: {all_band_names}")

# Save as GeoTIFF
stack.rio.to_raster(stack_file, dtype=np.float32)
print(f"\n✓ Saved: {stack_file}")
print("="*60)

## 4. Load WorldCover as Ground Truth
### WorldCover 2021 provides global land cover classification

In [ ]:
print("="*60)
print("LOADING WORLDCOVER 2021 AS GROUND TRUTH")
print("="*60)

if not os.path.exists(worldcover_file):
    raise FileNotFoundError(f"WorldCover file not found: {worldcover_file}")

# Load Sentinel-2 stack metadata for alignment
with rasterio.open(stack_file) as src:
    stack_transform = src.transform
    stack_shape = (src.height, src.width)
    stack_crs = src.crs
    stack_bounds = src.bounds

print(f"Stack dimensions: {stack_shape[0]}x{stack_shape[1]} pixels")
print(f"Stack CRS: {stack_crs}")

# Load and reproject WorldCover to match Sentinel-2 stack
with rasterio.open(worldcover_file) as src:
    from rasterio.warp import reproject, Resampling
    
    worldcover_data = np.empty(stack_shape, dtype=np.uint8)
    
    reproject(
        source=rasterio.band(src, 1),
        destination=worldcover_data,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=stack_transform,
        dst_crs=stack_crs,
        resampling=Resampling.nearest
    )

# Convert WorldCover classes to binary green/non-green
# WorldCover legend:
# 10 = Tree cover
# 20 = Shrubland  
# 30 = Grassland
# 40 = Cropland
# 50 = Built-up
# 60 = Bare / sparse vegetation
# 70 = Snow and ice
# 80 = Permanent water bodies
# 90 = Herbaceous wetland
# 95 = Mangroves
# 100 = Moss and lichen

GREEN_CLASSES = [10, 20, 30, 95]  # Tree, Shrub, Grass, Mangroves
labels = np.isin(worldcover_data, GREEN_CLASSES).astype(np.uint8)

# Save WorldCover-derived labels
with rasterio.open(
    worldcover_labels_file, 'w',
    driver='GTiff',
    height=labels.shape[0],
    width=labels.shape[1],
    count=1,
    dtype=labels.dtype,
    crs=stack_crs,
    transform=stack_transform,
    compress='lzw'
) as dst:
    dst.write(labels, 1)

green_pixels = np.sum(labels == 1)
total_pixels = labels.size
green_percentage = (green_pixels / total_pixels) * 100

print(f"\n✓ WorldCover processed successfully")
print(f"  Green classes: {GREEN_CLASSES}")
print(f"  Green pixels: {green_pixels:,} ({green_percentage:.2f}%)")
print(f"  Non-green pixels: {total_pixels - green_pixels:,} ({100-green_percentage:.2f}%)")
print(f"✓ Saved: {worldcover_labels_file}")
print("="*60)

## 5. Prepare Training Data

In [ ]:
print("="*60)
print("PREPARING TRAINING DATA")
print("="*60)

# Load Sentinel-2 stack
with rasterio.open(stack_file) as src:
    X_stack = src.read()  # Shape: (21, 512, 512)

# Load WorldCover labels
with rasterio.open(worldcover_labels_file) as src:
    y = src.read(1)  # Shape: (512, 512)

print(f"Sentinel-2 stack shape: {X_stack.shape}")
print(f"WorldCover labels shape: {y.shape}")

# Reshape for sklearn: (n_samples, n_features)
n_bands = X_stack.shape[0]
n_pixels = X_stack.shape[1] * X_stack.shape[2]

X = X_stack.reshape(n_bands, -1).T  # Shape: (262144, 21)
y_flat = y.flatten()  # Shape: (262144,)

# Remove NaN values
valid_mask = ~np.isnan(X).any(axis=1)
X_clean = X[valid_mask]
y_clean = y_flat[valid_mask]

print(f"\nData after cleaning:")
print(f"  Valid pixels: {len(X_clean):,}")
print(f"  Green samples: {np.sum(y_clean == 1):,} ({100*np.sum(y_clean == 1)/len(y_clean):.2f}%)")
print(f"  Non-green samples: {np.sum(y_clean == 0):,} ({100*np.sum(y_clean == 0)/len(y_clean):.2f}%)")

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean, test_size=0.2, random_state=42, stratify=y_clean
)

print(f"\nTrain-test split:")
print(f"  Training samples: {len(X_train):,}")
print(f"  Testing samples: {len(X_test):,}")
print("="*60)

## 6. Train Random Forest Model
### Using WorldCover as ground truth

In [ ]:
print("="*60)
print("TRAINING RANDOM FOREST MODEL")
print("="*60)

# Initialize Random Forest
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=50,
    min_samples_leaf=20,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

print("Training Random Forest...")
rf.fit(X_train, y_train)

print("\n✓ Model trained successfully")
print("="*60)

## 7. Evaluate Model Performance

In [ ]:
print("="*60)
print("EVALUATING MODEL PERFORMANCE")
print("="*60)

# Make predictions
y_pred = rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"\nModel Performance (trained on WorldCover):")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"\nConfusion Matrix:")
print(cm)

# Save metrics
metrics = {
    "model": "RandomForest",
    "ground_truth": "WorldCover_2021",
    "accuracy": float(accuracy),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "confusion_matrix": cm.tolist()
}

with open(os.path.join(run_folder, "metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

print(f"\n✓ Metrics saved to: {run_folder}/metrics.json")
print("="*60)

## 8. Visualize Confusion Matrix

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Green', 'Green'],
            yticklabels=['Non-Green', 'Green'])
plt.title('Confusion Matrix - Random Forest (WorldCover GT)', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(os.path.join(run_folder, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✓ Confusion matrix saved")

## 9. Create Prediction Map

In [ ]:
print("="*60)
print("CREATING PREDICTION MAP")
print("="*60)

# Predict for entire image
y_pred_all = np.full(n_pixels, np.nan)
y_pred_all[valid_mask] = rf.predict(X_clean)

# Reshape to image dimensions
y_pred_map = y_pred_all.reshape(y.shape)

# Save prediction map
pred_file = os.path.join(run_folder, 'prediction_map.tif')
with rasterio.open(
    pred_file, 'w',
    driver='GTiff',
    height=y_pred_map.shape[0],
    width=y_pred_map.shape[1],
    count=1,
    dtype=np.float32,
    crs=stack_crs,
    transform=stack_transform,
    compress='lzw'
) as dst:
    dst.write(y_pred_map.astype(np.float32), 1)

print(f"✓ Prediction map saved: {pred_file}")
print(f"  Predicted green: {np.nansum(y_pred_map == 1)} pixels ({100*np.nansum(y_pred_map == 1)/np.sum(~np.isnan(y_pred_map)):.2f}%)")
print("="*60)

## 10. Feature Importance Analysis

In [ ]:
# Get feature importances
importances = rf.feature_importances_
band_names = [
    'B02-Apr', 'B03-Apr', 'B04-Apr', 'B08-Apr', 'NDVI-Apr', 'EVI-Apr', 'SAVI-Apr',
    'B02-Aug', 'B03-Aug', 'B04-Aug', 'B08-Aug', 'NDVI-Aug', 'EVI-Aug', 'SAVI-Aug',
    'B02-Nov', 'B03-Nov', 'B04-Nov', 'B08-Nov', 'NDVI-Nov', 'EVI-Nov', 'SAVI-Nov'
]

# Sort by importance
indices = np.argsort(importances)[::-1]

# Plot feature importances
plt.figure(figsize=(12, 8))
plt.barh(range(len(importances)), importances[indices])
plt.yticks(range(len(importances)), [band_names[i] for i in indices])
plt.xlabel('Feature Importance', fontsize=12)
plt.title('Random Forest Feature Importance (WorldCover GT)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(run_folder, 'feature_importance.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✓ Feature importance plot saved")
print("\nTop 5 most important features:")
for i in range(5):
    idx = indices[i]
    print(f"  {i+1}. {band_names[idx]}: {importances[idx]:.4f}")

## 11. Comparison Visualization
### Compare WorldCover GT, RF Prediction, and RGB

In [ ]:
# Create comparison visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 1. RGB composite (August)
rgb_indices = [8, 9, 10]  # B02-Aug, B03-Aug, B04-Aug
rgb = X_stack[rgb_indices, :, :].transpose(1, 2, 0)
rgb_norm = np.clip(rgb / 3000, 0, 1)
axes[0].imshow(rgb_norm)
axes[0].set_title("RGB (August Sentinel-2)", fontsize=14, fontweight='bold')
axes[0].axis('off')

# 2. WorldCover Ground Truth
axes[1].imshow(y, cmap='RdYlGn', vmin=0, vmax=1)
axes[1].set_title(f"WorldCover 2021 Ground Truth\nGreen: {100*y.sum()/y.size:.1f}%", 
                  fontsize=14, fontweight='bold')
axes[1].axis('off')

# 3. Random Forest Prediction
axes[2].imshow(y_pred_map, cmap='RdYlGn', vmin=0, vmax=1)
axes[2].set_title(f"Random Forest Prediction\nGreen: {100*np.nansum(y_pred_map==1)/np.sum(~np.isnan(y_pred_map)):.1f}%", 
                  fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.suptitle(f'{CITY} - Green Space Detection (WorldCover-trained)', 
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.savefig(os.path.join(run_folder, 'comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✓ Comparison visualization saved")

## 12. Summary Report

In [ ]:
print("\n" + "="*70)
print(f"SYDNEY GREEN SPACE DETECTION - SUMMARY REPORT")
print("="*70)
print(f"\nGround Truth: WorldCover 2021")
print(f"Green Classes: Tree cover (10), Shrubland (20), Grassland (30), Mangroves (95)")
print(f"\nModel Performance:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"\nGreen Coverage:")
print(f"  WorldCover GT: {100*y.sum()/y.size:.2f}%")
print(f"  RF Prediction: {100*np.nansum(y_pred_map==1)/np.sum(~np.isnan(y_pred_map)):.2f}%")
print(f"\nOutput Files:")
print(f"  Results folder: {run_folder}")
print(f"  - metrics.json")
print(f"  - confusion_matrix.png")
print(f"  - prediction_map.tif")
print(f"  - feature_importance.png")
print(f"  - comparison.png")
print("="*70)